# Projeto 1 - Ciência dos Dados

Nome: Gustavo Marques Borges 

Nome: Luiz Eduardo Correa Santoro

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [14]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import nltk
from nltk.tokenize import TweetTokenizer
import re 
# nltk.download('stopwords') dowlod stopword Corpus

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [15]:
import os

filename = 'Magazine_Luiza_Final.xlsx'
if filename in os.listdir():
   print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
   print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo Magazine_Luiza_Final.xlsx, tudo certo para prosseguir com a prova!


Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [16]:
train = pd.read_excel(filename, coverters={'Tweets': str})
train['Classificação'] = train['Classificação'].astype('category')
train.Classificação.cat.categories = ['Muito Irrelevante','Irrelevante','Nulo','Relevante','Muito Relevante']
train.Classificação.cat = pd.Categorical(train.Classificação, categories=['Muito Irrelevante','Irrelevante','Nulo','Relevante','Muito Relevante'], ordered=True)
train.head(5)

,Tweets,Classificação
0,@tavaresqq a sua troca deu certo? precisando é...,Muito Irrelevante
1,o pessoal da #magazineluiza ja me conhece de ...,Relevante
2,"@magazineluiza bom dia, cadê o meu retorno da ...",Muito Relevante
3,oi @magazineluiza o @luansantana tá há dois m...,Muito Irrelevante
4,@grondinellys @magazineluiza você só precisa d...,Irrelevante


In [17]:
filtra_Train_MI = train['Classificação'] == 'Muito Irrelevante'
Train_MI = train.loc[filtra_Train_MI,:] 
filtra_Train_I = train['Classificação'] == 'Irrelevante'
Train_I = train.loc[filtra_Train_I,:]
filtra_Train_N = train['Classificação'] == 'Nulo'
Train_N = train.loc[filtra_Train_N,:]
filtra_Train_R = train['Classificação'] == 'Relevante'
Train_R = train.loc[filtra_Train_R,:]
filtra_Train_MR = train['Classificação'] == 'Muito Relevante'
Train_MR = train.loc[filtra_Train_MR,:]
Train = [Train_MI,Train_I,Train_N,Train_R,Train_MR]
Train_MI

,Tweets,Classificação
0,@tavaresqq a sua troca deu certo? precisando é...,Muito Irrelevante
3,oi @magazineluiza o @luansantana tá há dois m...,Muito Irrelevante
6,@mateus_didi @americanascom @magazineluiza @re...,Muito Irrelevante
12,"rt @heartilyshawn: olá, @magazineluiza o canto...",Muito Irrelevante
14,@odeiopepino @detalhesslr @americanascom @maga...,Muito Irrelevante
...,...,...
741,"@sandrac58953439 oi, sandra. sabia que agora t...",Muito Irrelevante
743,@magazineluiza oi luciana legal lugar como voc...,Muito Irrelevante
744,@magazineluiza magalu vc é muito gostosa,Muito Irrelevante
747,@elisavitorio oi. peço desculpa por isso. 😔 me...,Muito Irrelevante


In [18]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test['Classificação'] = test['Classificação'].astype('category')
test.Classificação.cat.categories = ['Muito Irrelevante','Irrelevante','Nulo','Relevante','Muito Relevante']
test.Classificação.cat = pd.Categorical(test.Classificação, categories=['Muito Irrelevante','Irrelevante','Nulo','Relevante','Muito Relevante'], ordered=True)
test.head(5)

,Tweets,Classificação
0,@magazineluiza aquele que eh quero twr e nao t...,Nulo
1,@teeeeeeiii @magazineluiza kkkkk o coração qua...,Irrelevante
2,@onofrifernando oi! te chamei no dm da uma olh...,Muito Irrelevante
3,@felliperadama @americanascom @magazineluiza @...,Irrelevante
4,@robrtplant @magazineluiza bom dia! informamos...,Nulo


In [19]:
filtra_Test_MI = test['Classificação'] == 'Muito Irrelevante'
Test_MI = test.loc[filtra_Test_MI,:] 
filtra_Test_I = train['Classificação'] == 'Irrelevante'
Test_I = test.loc[filtra_Test_I,:]
filtra_Test_N = train['Classificação'] == 'Nulo'
Test_N = test.loc[filtra_Test_N,:]
filtra_Test_R = train['Classificação'] == 'Relevante'
Test_R = train.loc[filtra_Test_R,:]
filtra_Test_MR = train['Classificação'] == 'Muito Relevante'
Test_MR = test.loc[filtra_Test_MR,:]
Test = [Test_I,Test_MI,Test_N,Test_R,Test_MR]

___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Nosso produto é a **Magazine Luiza**, uma empresa varejista de eletrônicos e imóveis. Possui lojas físicas, mas tem maior notoriedade em sua plataforma digital, cujas críticas e elogios estão fortemente presente na conta do Tweeter da empresa.

Para a classificação dos tweets envolvendo a Magazine Luiza, consideramos:
   * **“muito relevantes”** aqueles que possuem problemas ou elogios à marca, detalhando o motivo de sua crítica 
   * **“relevantes”** os menos detalhados.
   * **“Nulo”** são aqueles que demandam cupons de desconto para a marca e tweets com respostas à própria marca, que aparecem, na maioria das vezes, sem contexto. 
   * **“irrelevantes”** foram os comentários ofensivos, que mostravam indignação com a marca, mas utilizando apenas palavras de baixo calão.
   * **“Muito irrelevante”** foi marcado para os tweets que a própria marca fazia, respondendo os clientes ou para os tweets que não tinham nada a ver com a marca tampouco com o que ela propõe.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [22]:
#define a funcao para tratar os dados 
def cleanup(tweet):
    #cria uma lista para a lista de palavaras filtradas
    tweet_limpo = []
    #definicao dos metodos de tratamento do dataframe
    punctuation = '[@!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    stopwords = nltk.corpus.stopwords.words('portuguese') 
    text_subbed = re.sub(pattern, '', tweet)
    #Limpa stopwords
    token_words = TweetTokenizer().tokenize(text_subbed)
    for word in token_words:
        if word not in stopwords:
            tweet_limpo.append(word)
    return tweet_limpo

In [28]:
#Limpa os tweets de acordo com  a sua relevancia 
Train_Limpo = []
for dataframe in Train:
    lista_de_tweets = dataframe['Tweets'].values.tolist() 
    lista_tweets_limpos =[]
    for tweet in lista_de_tweets:
        train_TwL = cleanup(tweet)
        lista_tweets_limpos.extend(train_TwL)
    Train_Limpo.append(lista_tweets_limpos)
    
#Caucula e aloca em variavel a probabibidade de ser relevante
prob_Re = train['Classificação'].value_counts(True, sort=False)
P_MI = prob_Re[0]
P_I = prob_Re[1]
P_N = prob_Re[2]
P_R = prob_Re[3]
P_MR = prob_Re[4]

P_Re = [P_MI,P_I,P_N,P_R,P_MR]

#Separo as listas provevineites dos dataframes
palavras_train_MI = pd.Series(Train_Limpo[0]).value_counts().to_dict()
palavras_train_I = pd.Series(Train_Limpo[1]).value_counts().to_dict()
palavras_train_N = pd.Series(Train_Limpo[2]).value_counts().to_dict()
palavras_train_R = pd.Series(Train_Limpo[3]).value_counts().to_dict()
palavras_train_MR = pd.Series(Train_Limpo[4]).value_counts().to_dict()


dic_Fa_palavras=[palavras_train_MI, palavras_train_I,palavras_train_N,palavras_train_R,palavras_train_MR] 

In [52]:
def Naive_Bayes(tweet):
    clean_tweet = cleanup(tweet)
    serie_de_fa =[]
    for dic in dic_Fa_palavras:
        dic_f ={}
        for palavra in clean_tweet:
            if palavra in dic:
                dic_f[palavra] = dic[palavra]
            if palavra not in dic:
                dic_f[palavra] = 0
        serie_de_fa.append(pd.Series(dic_f))
        
    fa_palavras_MI = serie_de_fa[0]
    fa_palavras_I = serie_de_fa[1]
    fa_palavras_N = serie_de_fa[2]
    fa_palavras_R = serie_de_fa[3]
    fa_palavras_MR = serie_de_fa[4]
    fa_palavras_Re = [fa_palavras_MI,fa_palavras_I,fa_palavras_N,fa_palavras_R,fa_palavras_MR]  
    lista_prob = []
    alpha = 0.001
    j = 0
    for i in fa_palavras_Re:
        contagem_palavra_mais_alpha = i.add(alpha)
        contagem_todas_palavras = i.sum()
        laplace_smothing = contagem_palavra_mais_alpha.divide(contagem_todas_palavras + alpha*(i.size))
        smoothing = np.log10(laplace_smothing)
        P_palavras_dado_Re = smoothing.sum()
        lista_prob.append(P_palavras_dado_Re + (np.log10(P_Re[j])))
        j+=1
    MAX = 0
    for k in range(0,len(lista_prob)):
        if lista_prob[k] >= lista_prob[MAX]:
            MAX = k
    if MAX == 0:
        Relevancia_NV = 'Muito Irrelevante'
    if MAX == 1:
        Relevancia_NV = 'Irrelevante'
    if MAX == 2:
        Relevancia_NV = 'Nulo'
    if MAX == 3:
        Relevancia_NV = 'Relevante'
    if MAX == 4:
        Relevancia_NV = 'Muito Relevante'

    return Relevancia_NV

In [53]:
Cat_NaiveBayes = []
for i in train['Tweets']:
    Cat_NaiveBayes.append(Naive_Bayes(i))
train['Relevancia_NB'] = Cat_NaiveBayes

In [54]:
Relevancia_NB_X_Classificacao = pd.crosstab(train.Classificação,train.Relevancia_NB,normalize=True)
Relevancia_NB_X_Classificacao

Relevancia_NB,Irrelevante,Muito Irrelevante,Muito Relevante,Nulo,Relevante
Classificação,,,,,
Muito Irrelevante,0.002677,0.414993,0.001339,0.001339,0.001339
Irrelevante,0.160643,0.017403,0.001339,0.002677,0.001339
Nulo,0.001339,0.001339,0.001339,0.100402,0.004016
Relevante,0.000000,0.001339,0.005355,0.002677,0.183400
Muito Relevante,0.000000,0.000000,0.093708,0.000000,0.000000


In [56]:
print(f'A proporcao de acerto do nossvo classificador eh {(Relevancia_NB_X_Classificacao.iloc[1,0]+Relevancia_NB_X_Classificacao.iloc[0,1]+Relevancia_NB_X_Classificacao.iloc[4,3]+Relevancia_NB_X_Classificacao.iloc[3,3]+Relevancia_NB_X_Classificacao.iloc[3,4])*100:.5g}%')

A proporcao de acerto do nossvo classificador eh 76.171%



___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**